In [1]:
import scipy.io as sio
import numpy as np
import os
from utils import *
from tqdm import tqdm,trange

In [2]:
#从fmri数据和keys数据中取交集，找到共同keys部分数据
#对于每个个体进行如下操作：
    #首先从原始keys数据中读取边信息，再将frmi数据中为空的index去除，计算皮尔逊系数矩阵
    #计算功能相关性系数矩阵和结构的相连矩阵,分别保存成两个matlab矩阵
    #计算低秩稀疏矩阵，，matlab
    #利用低秩稀疏矩阵抽象为大脑网络


In [3]:
fmri_path="../matfile/res_fmri_data/111009_rfMRI_REST1_LR_Atlas.dtseries.mat"
data=sio.loadmat(fmri_path)["fmri_mat"]
# print(data.keys("fmri_mat"))
# print(data["fmri_mat"])
#统计有多少nan值
# print(np.isnan(data["fmri_mat"]).sum()/1200)


In [4]:
#求出所有个体
structual_path="../matfile/distance_4/"
function_path="../matfile/res_fmri_data/"

structual_keys=os.listdir(structual_path)
structual_keys=[i[:6]for i in structual_keys if i.endswith("keys.mat")] 
function_keys=os.listdir(function_path)
function_keys=[i[:6]for i in function_keys ]
keys_all=set(structual_keys).intersection(set(function_keys))
keys_all=list(keys_all)

In [29]:
key_test=keys_all[0]
INDEX_path="../matfile/surfaces/"
# INDEX=sio.loadmat(INDEX_path+key_test+"_INDEX.mat")["INDEX"]
keys_path="../matfile/distance_4/"
#拿到index标签数据
index_data=sio.loadmat(INDEX_path+key_test+"INDEX.mat")["INDEX"][0][0][0]
indexs=list(index_data[1:])
indexs
index_to_oringin_dic={}
oringin_to_index_dic={}
for i in range(len(indexs)):
    index_to_oringin_dic[i+1]=indexs[i]
    oringin_to_index_dic[indexs[i]]=i+1
#index 在连接矩阵中不一定拥有全部的值
print(len(indexs),len(index_to_oringin_dic))

6081 6081


In [30]:
fmri_data=sio.loadmat(function_path+key_test+"_rfMRI_REST1_LR_Atlas.dtseries.mat")["fmri_mat"]
#取出的仅仅是取了index之后的那些数据
fmri_data=fmri_data[indexs]
fmri_data.shape


(6081, 1200)

In [31]:
#为空的数据在这部分需要重新去掉
remove_index=list(set(np.where(np.isnan(fmri_data))[0]))
remove_index=[i for i in remove_index]
remove_index.sort()
fmri_dic={}
for i in range(len(indexs)):
    fmri_dic[i]=indexs[i]

# print(remove_index)
remove_index_anti=[fmri_dic[i] for i in remove_index]
print(remove_index_anti)

[7, 22, 26, 30, 34, 38, 42, 45, 49, 53, 57, 60, 63, 66, 127, 129, 131, 133, 135, 137, 139, 142, 145, 147, 149, 151, 153, 156, 159, 304, 318, 321, 324, 328, 335, 338, 364, 401, 432, 436, 463, 471, 477, 491, 506, 530, 536, 574, 597, 600, 606, 612, 616, 630, 635, 671, 696, 702, 713, 725, 752, 765, 788, 793, 804, 840, 854, 864, 875, 908, 924, 931, 938, 951, 959, 966, 1013, 1024, 1032, 1046, 1074, 1078, 1135, 1152, 1156, 1175, 1180, 1217, 1273, 1310, 1334, 1357, 1413, 1416, 1485, 1819, 1823, 1826, 1829, 2520, 2603, 2674, 2682, 2748, 2785, 2854, 2859, 2919, 2999, 3007, 3054, 3057, 3085, 3129, 3132, 3144, 3147, 3155, 3207, 3214, 3226, 3239, 3244, 3249, 3261, 3266, 3284, 3290, 3298, 3304, 3311, 3323, 3350, 3354, 19832, 19834, 19840, 19844, 19848, 19861, 19865, 19869, 19874, 19880, 19886, 26500, 26599, 26699, 26786, 26790, 26882, 26928, 26959, 26964, 27009, 27022, 27039, 27053, 27059, 27082, 27087, 27129, 27138, 27142, 27155, 27160, 27170, 27183, 27243, 27247, 27262, 27269, 27273, 27291, 27317,

In [32]:
edge_list=load_edge(structual_path+key_test+"keys.mat")

edge_list=[(int(i[0]),int(i[1])) for i in edge_list]

# for item in tqdm(edge_list):
#     if item[0]  in remove_index_anti or item[1]  in remove_index_anti:
#         pass
#     else:
#          edge_new.append((int(item[0]),int(item[1])))

# for it in edge_list:
#     node_list.add(int(it[0]))
#     node_list.add(int(it[1]))
# node_list=list(node_list)
# # node_list
edge_list_new,node1,node2=reindex(edge_list)
print(len(node1))
structual_index=[index_to_oringin_dic[i] for i in node1]
# structual_index

5719


In [12]:
fmri_data=sio.loadmat(function_path+key_test+"_rfMRI_REST1_LR_Atlas.dtseries.mat")["fmri_mat"]
#取出的仅仅是取了index之后的那些数据
fmri_data=fmri_data[structual_index]
fmri_data.shape
#为空的数据在这部分需要重新去掉
remove_index=list(set(np.where(np.isnan(fmri_data))[0]))
remove_index=[i+1 for i in remove_index]
remove_index.sort()
# print(remove_index)
remove_index_anti=[index_to_oringin_dic[i] for i in remove_index]
print(remove_index_anti)
#去除这部分的空值index



[7, 23, 28, 32, 36, 39, 43, 47, 50, 54, 100, 104, 107, 111, 118, 126, 128, 131, 133, 135, 137, 151, 154, 157, 164, 169, 172, 175, 280, 302, 308, 312, 315, 323, 327, 345, 390, 401, 423, 435, 455, 461, 469, 490, 514, 527, 534, 548, 580, 587, 604, 654, 660, 677, 694, 709, 715, 719, 734, 749, 786, 802, 809, 813, 832, 838, 848, 867, 883, 904, 917, 951, 959, 971, 977, 1024, 1050, 1105, 1114, 1766, 1769, 1772, 1779, 1782, 2124, 2284, 2367, 2414, 2421, 2496, 2521, 2532, 2675, 2683, 2700, 2749, 2822, 2827, 2866, 2875, 2888, 2910, 2930, 2950, 2955, 2962, 2971, 2993, 3016, 3030, 3034, 3064, 3070, 3086, 3091, 3099, 3105, 3124, 3139, 19193, 19213, 19237, 19248, 19261, 19265, 19299, 19304, 20213, 25061, 25155, 25188, 25191, 25214, 25231, 25255, 25277, 25281, 25283, 25285, 25302, 25307, 25310, 25313, 25342, 25348, 25362, 25368, 25387, 25394, 25397, 25407, 25417, 25423, 25445, 25450, 25453, 25470, 25473, 25477, 25481, 25499, 25503, 25507, 25525, 25528, 25531, 25533, 25552, 25556, 25559, 25577, 25580, 

In [1]:
all_index=list(set(structual_index)-set(remove_index_anti))
all_index.sort()
function_matrix=None
print(all_index[:50])
print(len( all_index))

NameError: name 'structual_index' is not defined

In [34]:
fmri_data_new=sio.loadmat(function_path+key_test+"_rfMRI_REST1_LR_Atlas.dtseries.mat")["fmri_mat"]
# fmri_data=fmri_data[all_index]
functional_matrix=fmri_data_new[all_index]
print(np.where(np.isnan(functional_matrix))[0])

[]


In [20]:
functional_matrix
fun_corr=np.corrcoef(functional_matrix)
fun_corr.shape

(5308, 5308)

In [66]:
structual_matrix=np.zeros((len(all_index),len(all_index)))

edge_list=load_edge(structual_path+key_test+"keys.mat")
edge_list=[(int(i[0]),int(i[1])) for i in edge_list]
#从原始数据中获取 



structual_matrix.shape


(5297, 5297)

In [67]:
for edge in tqdm(edge_list):
    if index_to_oringin_dic[edge[0]] in all_index and index_to_oringin_dic[edge[1]] in all_index:
        # print("ss")
        index_matrix_x=all_index.index(index_to_oringin_dic[edge[0]])
        index_matrix_y=all_index.index(index_to_oringin_dic[edge[1]])
        structual_matrix[index_matrix_x,index_matrix_y]=1
        structual_matrix[index_matrix_y,index_matrix_x]=1


100%|██████████| 98730/98730 [00:20<00:00, 4815.60it/s] 


In [70]:
# np.savetxt("structual_matrix.txt",structual_matrix,fmt="%d")

unique,count=np.unique(structual_matrix,return_counts=True)
data_count=dict(zip(unique,count))
data_count

{0.0: 27979167, 1.0: 79042}

In [71]:
sio.savemat("structual_matrix.mat",{"structual_matrix":structual_matrix})
sio.savemat("functional_matrix.mat",{"functional_matrix":fun_corr})

In [3]:
result_data=sio.loadmat("result.mat")["A"]
result_data

array([[0.00000000e+00, 0.00000000e+00, 6.80103491e-04, ...,
        1.51753477e-03, 0.00000000e+00, 7.89399757e-05],
       [0.00000000e+00, 0.00000000e+00, 9.88624336e-04, ...,
        3.19641116e-06, 4.24672166e-05, 2.04327236e-04],
       [6.80103491e-04, 9.88624336e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 9.62380988e-04],
       ...,
       [1.51753477e-03, 3.19641116e-06, 0.00000000e+00, ...,
        0.00000000e+00, 2.99482491e-02, 4.44113893e-03],
       [0.00000000e+00, 4.24672166e-05, 0.00000000e+00, ...,
        2.99482491e-02, 0.00000000e+00, 4.81601328e-02],
       [7.89399757e-05, 2.04327236e-04, 9.62380988e-04, ...,
        4.44113893e-03, 4.81601328e-02, 0.00000000e+00]])

In [14]:
index=np.where(result_data>0.01)
print(len(index[0]))

130030


In [36]:
import scipy.io as sio
import numpy as np
import os
from utils import *
from tqdm import tqdm,trange

structual_path="../matfile/distance_4/"
function_path="../matfile/res_fmri_data/"

structual_keys=os.listdir(structual_path)
structual_keys=[i[:6]for i in structual_keys if i.endswith("keys.mat")] 
function_keys=os.listdir(function_path)
function_keys=[i[:6]for i in function_keys ]
keys_all=set(structual_keys).intersection(set(function_keys))
keys_all=list(keys_all)

def save_all_index(index,name):
    save_path="../matfile/functional_and_structual_matrix/"
    with open(save_path+name+"all_inde.txt",'w') as f:
        for i in index:
            f.write(str(i)+'\n')





key_test=keys_all[0]
INDEX_path="../matfile/surfaces/"
# INDEX=sio.loadmat(INDEX_path+key_test+"_INDEX.mat")["INDEX"]
keys_path="../matfile/distance_4/"
#拿到index标签数据
index_data=sio.loadmat(INDEX_path+key_test+"INDEX.mat")["INDEX"][0][0][0]
indexs=list(index_data[1:])
index_to_oringin_dic={}
oringin_to_index_dic={}
for i in range(len(indexs)):
    index_to_oringin_dic[i+1]=indexs[i]
    oringin_to_index_dic[indexs[i]]=i+1

#去除structual index
fmri_data=sio.loadmat(function_path+key_test+"_rfMRI_REST1_LR_Atlas.dtseries.mat")["fmri_mat"]
#取出的仅仅是取了index之后的那些数据
fmri_data=fmri_data[indexs]
fmri_data.shape

#求需要去除的nan对应的index
remove_index=list(set(np.where(np.isnan(fmri_data))[0]))
remove_index=[i for i in remove_index]
remove_index.sort()
fmri_dic={}
for i in range(len(indexs)):
    fmri_dic[i]=indexs[i]

# print(remove_index)
remove_index_anti=[fmri_dic[i] for i in remove_index]
# print(remove_index_anti)


edge_list=load_edge(structual_path+key_test+"keys.mat")
edge_list=[(int(i[0]),int(i[1])) for i in edge_list]
edge_list_new,node1,node2=reindex(edge_list)
structual_index=[index_to_oringin_dic[i] for i in node1]

all_index=list(set(structual_index)-set(remove_index_anti))
all_index.sort()
print("最终保存的节点数："+str(len(all_index)))
save_all_index(all_index,key_test)
#读取fmri数据并求出corr
fmri_data_new=sio.loadmat(function_path+key_test+"_rfMRI_REST1_LR_Atlas.dtseries.mat")["fmri_mat"]
# fmri_data=fmri_data[all_index]
functional_matrix=fmri_data_new[all_index]
functional_matrix
fun_corr=np.corrcoef(functional_matrix)
#计算structual数据
structual_matrix=np.zeros((len(all_index),len(all_index)))
edge_list=load_edge(structual_path+key_test+"keys.mat")
edge_list=[(int(i[0]),int(i[1])) for i in edge_list]
for edge in tqdm(edge_list,desc="structual"):
    if index_to_oringin_dic[edge[0]] in all_index and index_to_oringin_dic[edge[1]] in all_index:
        # print("ss")
        index_matrix_x=all_index.index(index_to_oringin_dic[edge[0]])
        index_matrix_y=all_index.index(index_to_oringin_dic[edge[1]])
        structual_matrix[index_matrix_x,index_matrix_y]=1
        structual_matrix[index_matrix_y,index_matrix_x]=1

#保存数据
save_path="../matfile/functional_and_structual_matrix/"
sio.savemat(f"{save_path}{key_test}structual_matrix.mat",{"structual_matrix":structual_matrix})
sio.savemat(f"{save_path}{key_test}functional_matrix.mat",{"functional_matrix":fun_corr})

最终保存的节点数：5413


structual: 100%|██████████| 95446/95446 [00:20<00:00, 4739.68it/s]


In [37]:
print(np.where(np.isnan(functional_matrix)))

print(len(functional_matrix))

(array([], dtype=int64), array([], dtype=int64))
5413


In [6]:

def load_matrix(path):
    data=sio.loadmat(path)["A"]
    return data


In [17]:
print(len(np.where(data>0.01)[0]))


189570


In [1]:
keys_all=os.listdir("../matfile/result_low_matrix/")
keys_all=[i[:6]for i in keys_all if i.endswith("mat")]

In [2]:
def get_edge_list_from_matrix(matrix):
    edge_list=[]
    for i in range(len(matrix)):
        for j in range(i,len(matrix)):
            if matrix[i][j]>0.01 :
                edge_list.append((i,j))
    return edge_list
def save_edges_to_txt(edge_list,name):
    save_path="../graph_file/result_low_matrix/"
    with open(save_path+name+".txt",'w') as f:
        for i in edge_list:
            f.write(str(i[0])+" "+str(i[1])+'\n')


In [11]:
path="../matfile/result_low_matrix/"

for key in tqdm(keys_all):
      
    data=load_matrix(path+key+".mat")
    data.shape
    edges=get_edge_list_from_matrix(data)
    edge_new,dic1,dic2=reindex(edges)
    # save_txt_for_edges(edge_new,key+"_new.txt")

    save_edges_to_txt(edge_new,key)

100%|██████████| 39/39 [04:53<00:00,  7.54s/it]


In [3]:
def load_edge_from_txt(filename):
    with open(filename,'r') as f:
        edge_list=[]
        for line in f.readlines():
            edge_list.append(line.strip().split(" "))
    return edge_list


In [13]:
files=os.listdir("../graph_file/result_low_matrix/")
for file in files:
    edge_list=load_edge_from_txt("../graph_file/result_low_matrix/"+file)
    edge_list=[(int(i[0]),int(i[1])) for i in edge_list]
    edge_new,dic1,dic2=reindex(edge_list)
    save_txt_for_edges(edge_new,file[:-4]+"_new.txt")

In [9]:
save_txt_for_edges(edge_list,"115825.txt")

In [5]:
edge_new,dic1,dic2=reindex(edge_list)


In [ ]:
print(dic2)

In [5]:
import networkx as nx
G=nx.read_edgelist("../graph_file/result_low_matrix/115825.txt",nodetype=int)

In [31]:
#获取结果节点->由reindex计算index,还原原始index->读取index文件->
# 读取surface文件，获取节点对应坐标-》存入到文件中
def get_node_vtk_from_low_matrix(key):
    result_path="../graph_file/result_low_matrix/"
    oringin_graph_path="../graph_file/low_matrix/"
    index_path="../matfile/functional_and_structual_matrix/"
    surface_path="../matfile/surfaces/"


    # surface_data=sio.loadmat("../matfile/surface_data.mat")["surface_data"]
    edge_list=load_edge_from_txt(oringin_graph_path+key+"_new.txt")
    edge_list=[(int(i[0]),int(i[1])) for i in edge_list]
    _,_,dic2=reindex(edge_list)
    result_nodes=[]
    with open(result_path+key+"_new.txt",'r') as f:
        for line in f.readlines():
            result_nodes.append(dic2[int(line.strip())])
    index_low_matrix=[]
    with open(index_path+key+"all_inde.txt",'r') as f:
        for line in f.readlines():
            index_low_matrix.append(int(line.strip()))
    index_dic={}
    for i in range(len(index_low_matrix)):
        
        index_dic[i]=index_low_matrix[i]
    vertex_data=sio.loadmat(surface_path+key+"surface.mat")["surface"][0][0][0].T
    result_vertice=[]
    # print(result_nodes)
    for i in result_nodes[:200]:
        #此处-1是为了匹配matlab数据中的Index
        # index_dic[i]
        result_vertice.append(vertex_data[index_dic[i]-1])
    #将坐标保存到vtk
    save_path="../graph_file/result_low_matrix_vtk/"
    save_vtk(result_vertice,save_path+key+".vtk")


file=os.listdir("../graph_file/result_low_matrix/")
file=[i[:6] for i in file if i.endswith("txt")]
for key in file:
    get_node_vtk_from_low_matrix(key)    
    
# get_node_vtk_from_low_matrix("115825")








In [ ]:
#python实现从服务器远程下载文件
import paramiko
import os
from stat import S_ISDIR  as isdir
def down_from_remote(sftp_obj, remote_dir_name, local_dir_name):
    """远程下载文件"""
    remote_file = sftp_obj.stat(remote_dir_name)
    if isdir(remote_file.st_mode):
        # 文件夹，不能直接下载，需要继续循环
        check_local_dir(local_dir_name)
        print('开始下载文件夹：' + remote_dir_name)
        for remote_file_name in sftp.listdir(remote_dir_name):
            sub_remote = os.path.join(remote_dir_name, remote_file_name)
            sub_remote = sub_remote.replace('\\', '/')
            sub_local = os.path.join(local_dir_name, remote_file_name)
            sub_local = sub_local.replace('\\', '/')
            down_from_remote(sftp_obj, sub_remote, sub_local)
    else:
        # 文件，直接下载
        if remote_dir_name.endswith('input_surface.vtk'):

            print('开始下载文件：' + remote_dir_name)
            sftp_obj.get(remote_dir_name, local_dir_name)
        # print('开始下载文件：' + remote_dir_name)
        # sftp.get(remote_dir_name, local_dir_name)
 
def download_vtk_files(sftp_obj,remote_dir_name,local_dir_name):
    remote_file = sftp_obj.stat(remote_dir_name)
    
    dirs=sftp.listdir(remote_dir_name)
    for dir in dirs:
        try:
            file=sftp.listdir(os.path.join(remote_dir_name,dir))
            for f in file:
                
                if f.endswith('input_surface.vtk'):
                    try:
                        sftp_obj.get(remote_dir_name+"/"+dir+"/"+f, local_dir_name+dir+"_"+f)
                        print('开始下载文件：' + remote_dir_name+dir+f)
                    except:
                        pass
        except:
            pass
                    






def check_local_dir(local_dir_name):
    """本地文件夹是否存在，不存在则创建"""
    if not os.path.exists(local_dir_name):
        os.makedirs(local_dir_name)
 
 
if __name__ == "__main__":
    """程序主入口"""
    # 服务器连接信息
    host_name = '10.69.47.36'
    user_name = 'haiyang'
    password = '123456'
    port = 2222
    # 远程文件路径（需要绝对路径）
    remote_dir = '/disk1/wjunx/data/HCP900/'
    # 本地文件存放路径（绝对路径或者相对路径都可以）
    local_dir = '../matfile/surface_vtk_file/'

 
    # 连接远程服务器
    t = paramiko.Transport((host_name, port))
    t.connect(username=user_name, password=password)
    sftp = paramiko.SFTPClient.from_transport(t)
 
    # 远程文件开始下载
    # down_from_remote(sftp, remote_dir, local_dir)
    download_vtk_files(sftp,remote_dir,local_dir)
 
    # 关闭连接
    t.close()


In [ ]:
#寻找两个个体之间的相交index
def get_node_vtk_from_low_matrix(key):
    result_path="../graph_file/result_low_matrix/"
    oringin_graph_path="../graph_file/low_matrix/"
    index_path="../matfile/functional_and_structual_matrix/"
    surface_path="../matfile/surfaces/"

    # surface_data=sio.loadmat("../matfile/surface_data.mat")["surface_data"]
    edge_list=load_edge_from_txt(oringin_graph_path+key+".txt")
    edge_list=[(int(i[0]),int(i[1])) for i in edge_list]
    _,_,dic2=reindex(edge_list)
    result_nodes=[]
    with open(result_path+key+".txt",'r') as f:
        for line in f.readlines():
            result_nodes.append(dic2[int(line.strip())])
    index_low_matrix=[]
    with open(index_path+key+"all_inde.txt",'r') as f:
        for line in f.readlines():
            index_low_matrix.append(int(line.strip()))
    index_dic={}
    for i in range(len(index_low_matrix)):
        
        index_dic[i]=index_low_matrix[i]
    # vertex_data=sio.loadmat(surface_path+key+"surface.mat")["surface"][0][0][0].T
    # result_vertice=[]
    result_index=[]
    
    # print(result_nodes)
    for i in result_nodes[:200]:
        #此处-1是为了匹配matlab数据中的Index
        # index_dic[i]
        result_index.append(index_dic[i]-1)
    #将坐标保存到vtk
    return result_index
    # save_path="../graph_file/result_low_matrix_vtk/"
    # save_vtk(result_vertice,save_path+key+".vtk")

file=os.listdir("../graph_file/result_low_matrix/")
file=[i[:6] for i in file if i.endswith("txt")]
for key in  tqdm(file[:15]):
    for key1 in file[:15]:
    
        keys_oo=get_node_vtk_from_low_matrix(key)    
        key1=get_node_vtk_from_low_matrix(key1)
        #求两个集合交集的元素个数
        print(len(set(keys_oo).intersection(set(key1))))
        
    # print(keys_oo)




